---
## Simple generator example reading midi files
### Monday: 26-02-2024

* Read the MIDI file normally
    * We are given a list with all midi elements

* Read the MIDI file using the generator 
    * We are "`yield`ed" each element individually
    * We can get elements with `next()` or with a `loop`

---

In [6]:
#To help us import from outside folder
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))
from midiHandler import MidiHandler

midi_file_path = "../data/midi/test.mid"

midi = MidiHandler()

# Read the MIDI file normally
data = midi.readMidi(midi_file_path)
print(data)

# Read the MIDI file using the generator 
data = midi.readMidiGenerator(midi_file_path)
for d in data:
    print(d, end=' ')


[MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0), MetaMessage('set_tempo', tempo=500000, time=0), MetaMessage('track_name', name='Tempo Track', time=0), MetaMessage('track_name', name='New Instrument', time=0), Message('note_on', channel=0, note=92, velocity=25, time=0.7265625), Message('control_change', channel=0, control=64, value=29, time=0.09895833333333333), Message('control_change', channel=0, control=64, value=101, time=0.013020833333333332), Message('control_change', channel=0, control=64, value=124, time=0.015625), Message('control_change', channel=0, control=64, value=127, time=0.015625), Message('note_on', channel=0, note=88, velocity=18, time=0.3489583333333333), Message('note_off', channel=0, note=92, velocity=0, time=0.24739583333333331), Message('note_on', channel=0, note=83, velocity=33, time=0.26041666666666663), Message('note_off', channel=0, note=88, velocity=0, time=0.23958333333333331), Message('

---
## Lets get some random Midi sections and plot a few
### Friday: 01-03-2024

* Read the MIDI file normally
   * Handle 1 by 1 slowly

* Read the MIDI file using generators and threads
    * Use all cpu cores with yield generators
    
* Compare the differences
---